# GCP TTS API 

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
from google.cloud import texttospeech_v1 as tts
from dotenv import load_dotenv
import re

load_dotenv()

# 스크립트와 키워드
script = """이제 최초의 발표하는 인공지능 모델인 오인용에 대해 소개하겠습니다. 이 프로젝트는 중요한 내용을 중심으로 발표용 대본을 생성하는 기능을 가지고 있습니다. AI 음성 합성을 활용하여 자동으로 발표할 수 있는 기능을 제공함으로써, 기업, 연구자, 학생 등이 보다 효율적으로 정보를 활용할 수 있게 됩니다. 이 시스템은 일관된 발표 퀄리티를 유지하며, 발표에 드는 시간을 단축하는 기대 효과를 가지고 있습니다.

다음은 프로젝트 기획에 대해 설명드리겠습니다. 이 슬라이드에서는 프로젝트의 전반적인 기획 방향과 목표를 간략히 정리하겠습니다. 프로젝트의 기획은 체계적인 진행을 위해 매우 중요합니다."""
input_keywords = ["인공지능"]

# 문장 및 단어 추출
sentences = re.split(r'(?<=[.?!])\s+', script.strip())
tokenized_sentences = [re.findall(r'\w+', s) for s in sentences]
unique_words = sorted(set(word for sent in tokenized_sentences for word in sent))

# 임베딩
embedder = OpenAIEmbeddings()
word_embeddings = embedder.embed_documents(unique_words)
keyword_embeddings = [embedder.embed_query(k) for k in input_keywords]

# 유사도 기반 상위 3단어 추출 
similarities = {}
for word, w_emb in zip(unique_words, word_embeddings):
    sims = [cosine_similarity([w_emb], [k_emb])[0][0] for k_emb in keyword_embeddings]
    similarities[word] = max(sims)

top_emphasized_words = {word for word, _ in sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:3]}
print('강조할 단어 : ', top_emphasized_words)

# SSML 생성 (emphasis + prosody)
ssml_outputs = []
for idx, sent in enumerate(sentences):
    words = re.split(r'(\W+)', sent)
    processed = []
    for w in words:
        if w in top_emphasized_words:
            emphasized = (
                f'<prosody pitch="+10%" volume="+2dB">'
                f'<emphasis level="strong">{w}!</emphasis>'
                f'</prosody>'
            )
            processed.append(emphasized)
        else:
            processed.append(w)
    ssml = f"<speak>{''.join(processed).strip()}</speak>"
    ssml_outputs.append((ssml, f"output_{idx}.wav"))

# Google TTS 요청
client = tts.TextToSpeechClient()
voice = tts.VoiceSelectionParams(language_code="ko-KR", name="ko-KR-Standard-B") 
audio_config = tts.AudioConfig(audio_encoding=tts.AudioEncoding.LINEAR16)

# WAV 생성
for ssml, filename in ssml_outputs:
    print(f"[SSML 미리보기]\n{ssml}\n→ {filename}\n")
    response = client.synthesize_speech(
        input=tts.SynthesisInput(ssml=ssml),
        voice=voice,
        audio_config=audio_config
    )
    with open(filename, "wb") as out:
        out.write(response.audio_content)
        print(f"{filename} 저장 완료\n")

강조할 단어 :  {'기획', '인공지능', 'AI'}
[SSML 미리보기]
<speak>이제 최초의 발표하는 <prosody pitch="+10%" volume="+2dB"><emphasis level="strong">인공지능!</emphasis></prosody> 모델인 오인용에 대해 소개하겠습니다.</speak>
→ output_0.wav

output_0.wav 저장 완료

[SSML 미리보기]
<speak>이 프로젝트는 중요한 내용을 중심으로 발표용 대본을 생성하는 기능을 가지고 있습니다.</speak>
→ output_1.wav

output_1.wav 저장 완료

[SSML 미리보기]
<speak><prosody pitch="+10%" volume="+2dB"><emphasis level="strong">AI!</emphasis></prosody> 음성 합성을 활용하여 자동으로 발표할 수 있는 기능을 제공함으로써, 기업, 연구자, 학생 등이 보다 효율적으로 정보를 활용할 수 있게 됩니다.</speak>
→ output_2.wav

output_2.wav 저장 완료

[SSML 미리보기]
<speak>이 시스템은 일관된 발표 퀄리티를 유지하며, 발표에 드는 시간을 단축하는 기대 효과를 가지고 있습니다.</speak>
→ output_3.wav

output_3.wav 저장 완료

[SSML 미리보기]
<speak>다음은 프로젝트 기획에 대해 설명드리겠습니다.</speak>
→ output_4.wav

output_4.wav 저장 완료

[SSML 미리보기]
<speak>이 슬라이드에서는 프로젝트의 전반적인 <prosody pitch="+10%" volume="+2dB"><emphasis level="strong">기획!</emphasis></prosody> 방향과 목표를 간략히 정리하겠습니다.</speak>
→ output_5.wav

output_5.wav 저장 완료

[SSML 미리보기]
<speak>프로젝트의 기획은 체계적인 진행을 위해 매우 

할 일 : 전체 대본을 JSON으로 키워드와 함꼐 입력받고 전체 대본에서 유사어 10개를 JSON으로 반환

# Youtube 영상에서 WAV 추출

In [ ]:
import yt_dlp
from pydub import AudioSegment
import os

def time_convert(time_str):
    minutes = time_str // 100  
    seconds = time_str % 100   
    return minutes * 60 + seconds

def download_audio(url, save_dir, clip_idx, start=None, end=None):
    os.makedirs(save_dir, exist_ok=True)

    # 확장자 포함된 yt-dlp 저장용 임시 파일 경로
    temp_template = os.path.join(save_dir, f"CTS_temp_{clip_idx}.%(ext)s")
    temp_wav = os.path.join(save_dir, f"CTS_temp_{clip_idx}.wav")

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': temp_template,
        'quiet': True,
        'force_ipv4': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    # 자르고 저장
    audio = AudioSegment.from_file(temp_wav, format="wav")
    if start and end:
        start_ms = time_convert(start) * 1000
        end_ms = time_convert(end) * 1000
        audio = audio[start_ms:end_ms]

    # 최종 파일명: cheo_1.wav, cheo_2.wav ...
    final_path = os.path.join(save_dir, f"CTS_{clip_idx}.wav")
    audio.export(final_path, format="wav")

    os.remove(temp_wav)
    return audio

In [ ]:
url = "https://youtu.be/tRn4W5n39U8?si=EGa2PYHHboa2xajA&t=481"
save_dir = "../../data/test_wav"
audio_data = []
start = 0
end = 20
i = 1 

for time in range(480,40000,30):
    start += time
    end += time
    audio = download_audio(
        url = url,
        save_dir=save_dir,
        clip_idx=i,
        start=start,
        end=end
    )
    i += 1
    audio_data.append(audio)

# ZONOS

In [9]:
import torch
torch.cuda.empty_cache()

In [ ]:
import os
import torchaudio
from zonos.model import Zonos
from zonos.conditioning import make_cond_dict
import torch._dynamo  # suppress warning if needed

# ❗ phonemizer용 환경변수 설정 (espeak.dll 대응)
os.environ["PATH"] += os.pathsep + r"C:\Program Files\eSpeak NG"
os.environ["PHONEMIZER_ESPEAK_LIBRARY"] = r"C:\Program Files\eSpeak NG\espeak.dll"

# ❗ torch compile 비활성화 (C++ 컴파일러 없이 실행)
torch._dynamo.config.suppress_errors = True

c:\Users\user\Documents\GitHub\zonos\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 경로 설정
path = "../../data/test_wav"

# 모델 불러오기
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Zonos.from_pretrained("Zyphra/Zonos-v0.1-transformer", device=device)


In [ ]:
import os
import torch
import torchaudio
import librosa
import numpy as np

# 입력 음성 로드
wav, sampling_rate = torchaudio.load(os.path.join(path, "ma_1_test.wav"))

# 스피커 임베딩 생성
speaker = model.make_speaker_embedding(wav, sampling_rate)

text = """
현대 사회에서 *AI* 기술은 빠르게 발전하고 있습니다.
우리의 '발표' 능력을 지원하는 시스템이 필요합니다.
"""

cond_dict = make_cond_dict(text=text, speaker=speaker, language="ko")
conditioning = model.prepare_conditioning(cond_dict)

codes = model.generate(conditioning, disable_torch_compile=True)
wavs = model.autoencoder.decode(codes).cpu()

# 원본 저장
torchaudio.save(os.path.join(path, "sample.wav"), wavs[0], model.autoencoder.sampling_rate)
print("🔊 sample.wav 생성 완료!")

# # 후처리
# sr = model.autoencoder.sampling_rate
# audio_np = wavs[0].squeeze().numpy()  # [1, N] → [N] 보장

# # 1.3배 빠르게
# y_fast = librosa.effects.time_stretch(audio_np, rate=1.3)

# # +2 반음
# y_shifted = librosa.effects.pitch_shift(y_fast, sr=sr, n_steps=2)

# # shape 조정
# if y_shifted.ndim == 1:
#     y_shifted = np.expand_dims(y_shifted, axis=0)  # → [1, samples]

# # numpy → torch
# y_tensor = torch.from_numpy(y_shifted).float()  # float32로 맞춤

# # 저장
# output_path = os.path.join(path, "fast_pitchup.wav")
# torchaudio.save(output_path, y_tensor, sr)

# print("⚡ 1.3배 빠르게 + 🎵 피치 +2반음 적용 완료!")
# print(f"📁 저장 위치: {output_path}")


In [ ]:

def zero_shot(path,file,text,name, model):
    torch.cuda.empty_cache()
    wav, sampling_rate = torchaudio.load(os.path.join(path, file))
    speaker = model.make_speaker_embedding(wav, sampling_rate)

    cond_dict = make_cond_dict(
        text = text,
        speaker = speaker,
        language = "ko"
    )
    conditioning = model.prepare_conditioning(cond_dict)
    codes = model.generate(conditioning, disable_torch_compile=True)
    wavs = model.autoencoder.decode(codes).cpu()

    # 결과 저장
    torchaudio.save(os.path.join(path, f"{name}_zero_shot.wav"), wavs[0], model.autoencoder.sampling_rate)

    # print("🔊 zero_shot.wav 생성 완료!")
    # sr = model.autoencoder.sampling_rate
    # audio_np = wavs[0].numpy()

    # # [1] 1.3배 속도 (tempo)
    # y_fast = librosa.effects.time_stretch(audio_np, rate=1.3)

    # # [2] +2 반음 pitch up
    # y_shifted = librosa.effects.pitch_shift(y_fast, sr=sr, n_steps=2)

    # # 저장
    # final_out_path = os.path.join(path, f"{name}_fast_pitchup.wav")
    # sf.write(final_out_path, y_shifted, sr)


def few_shot(path, data, text, name, model):
    '''few-shot 보이스 클리닝'''
    torch.cuda.empty_cache()

    embeddings = []
    for file in data:
        wav, sampling_rate = torchaudio.load(os.path.join(path, file))
        emb = model.make_speaker_embedding(wav, sampling_rate)
        embeddings.append(emb)

    speaker_embedding = torch.stack(embeddings).mean(dim=0)

    cond_dict = make_cond_dict(
        text = text,
        speaker = speaker_embedding,
        language="ko"
    )
    conditioning = model.prepare_conditioning(cond_dict)
    # 음성 생성 (컴파일러 비활성화)
    codes = model.generate(conditioning, disable_torch_compile=True)
    wavs = model.autoencoder.decode(codes).cpu()

    # 결과 저장
    torchaudio.save(os.path.join(path, f"{name}_few_shot.wav"), wavs[0], model.autoencoder.sampling_rate)

    print("🔊 few_shot.wav 생성 완료!")
    return codes


In [ ]:
path = "../../data/test_wav"
file = "ma_1_test.wav"
data = [f"karina_{i}.wav" for i in range(1,5)]
name = "test01"
text = """
이러한 문제를 해결하기 위해, AI 기반 발표 지원 시스템인 "저희 발표 안합니다!" 프로젝트가 기획되었습니다.
"""

zero_shot(path,file, text,name, model)
# few_shot(path,data, text,name)

In [1]:
import os
import torch
import torchaudio
from zonos.model import Zonos
from zonos.conditioning import make_cond_dict
import time
import pandas as pd
from tabulate import tabulate

# 🔧 환경 설정
os.environ["PATH"] += os.pathsep + r"C:\Program Files\eSpeak NG"
os.environ["PHONEMIZER_ESPEAK_LIBRARY"] = r"C:\Program Files\eSpeak NG\espeak.dll"
torch._dynamo.config.suppress_errors = True

# 📦 모델 로딩
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Zonos.from_pretrained("Zyphra/Zonos-v0.1-transformer", device=device)

# 📁 경로 설정
TEXT_PATH = "../../data/txt/test_script.txt"
AUDIO_PATH = "../../data/test_wav"
os.makedirs(AUDIO_PATH, exist_ok=True)

# 🔊 참조 음성
zero_shot_ref = "CTS_3.wav"
few_shot_refs = ["CTS_4.wav", "CTS_5.wav"]

# 📄 텍스트 로드 및 페이지 분할
with open(TEXT_PATH, encoding="utf-8") as f:
    script = f.read()
pages = script.strip().split("\n\n")

# ✅ speaker 임베딩 캐싱 (단 1회)
zero_wav, zero_sr = torchaudio.load(os.path.join(AUDIO_PATH, zero_shot_ref))
zero_shot_speaker = model.make_speaker_embedding(zero_wav, zero_sr)

few_embeddings = []
for file in few_shot_refs:
    wav, sr = torchaudio.load(os.path.join(AUDIO_PATH, file))
    emb = model.make_speaker_embedding(wav, sr)
    few_embeddings.append(emb)
few_shot_speaker = torch.stack(few_embeddings).mean(dim=0)

# 🎙 Zero-Shot 합성
def zero_shot(text, name):
    torch.cuda.empty_cache()
    cond = make_cond_dict(text=text, speaker=zero_shot_speaker, language="ko")
    conditioning = model.prepare_conditioning(cond)
    codes = model.generate(conditioning, disable_torch_compile=True)
    wavs = model.autoencoder.decode(codes).cpu()
    torchaudio.save(os.path.join(AUDIO_PATH, f"{name}_zero_shot.wav"), wavs[0], model.autoencoder.sampling_rate)

# 🎙 Few-Shot 합성
def few_shot(text, name):
    torch.cuda.empty_cache()
    cond = make_cond_dict(text=text, speaker=few_shot_speaker, language="ko")
    conditioning = model.prepare_conditioning(cond)
    codes = model.generate(conditioning, disable_torch_compile=True)
    wavs = model.autoencoder.decode(codes).cpu()
    torchaudio.save(os.path.join(AUDIO_PATH, f"{name}_few_shot.wav"), wavs[0], model.autoencoder.sampling_rate)

c:\Users\user\Documents\GitHub\zonos\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 🕒 합성 실행 및 시간 측정
results = []
for idx, text in enumerate(pages):
    
    page_name = f"page_{idx}"
    print(f"\n🎤 처리 중: {page_name}")

    # Zero-Shot
    start = time.time()
    try:
        zero_shot(text, page_name)
        zero_time = round(time.time() - start, 2)
        print(f"✅ Zero-Shot 완료: {zero_time}s")
    except Exception as e:
        zero_time = f"Error: {str(e)}"

    # Few-Shot
    start = time.time()
    try:
        few_shot(text, page_name)
        few_time = round(time.time() - start, 2)
        print(f"✅ Few-Shot 완료: {few_time}s")
    except Exception as e:
        few_time = f"Error: {str(e)}"

    results.append({
        "page": page_name,
        "zero_shot": zero_time,
        "few_shot": few_time
    })

# 📊 결과 출력
df = pd.DataFrame(results)
print("\n⏱️ 페이지별 생성 시간:\n")
print(tabulate(df, headers='keys', tablefmt='github', showindex=False))


🎤 처리 중: page_0


Generating: 100%|██████████| 2588/2588 [01:53<00:00, 22.88it/s]


✅ Zero-Shot 완료: 143.52s


Generating: 100%|██████████| 2588/2588 [02:03<00:00, 21.03it/s]


✅ Few-Shot 완료: 125.55s

🎤 처리 중: page_1


...

✅ Few-Shot 완료: 133.79s

🎤 처리 중: page_15


Generating:  95%|█████████▌| 2470/2588 [01:53<00:05, 21.85it/s]


✅ Zero-Shot 완료: 122.51s


Generating:  75%|███████▍  | 1933/2588 [01:19<00:27, 24.24it/s]


✅ Few-Shot 완료: 80.61s

⏱️ 페이지별 생성 시간:

| page    |   zero_shot |   few_shot |
|---------|-------------|------------|
| page_0  |      143.52 |     125.55 |
| page_1  |      134.99 |     119.78 |
| page_2  |       79.26 |      89.5  |
| page_3  |      127.08 |     126.37 |
| page_4  |      137.22 |     136.45 |
| page_5  |      125.05 |     129.81 |
| page_6  |       73.77 |      66.38 |
| page_7  |      137.52 |     135.93 |
| page_8  |       73.29 |      81.02 |
| page_9  |      124.1  |     133.46 |
| page_10 |      128.26 |     128.11 |
| page_11 |      124.38 |     132.2  |
| page_12 |      129.16 |     126.78 |
| page_13 |       77.17 |     131.24 |
| page_14 |      124.6  |     133.79 |
| page_15 |      122.51 |      80.61 |


In [ ]:
# ✅ speaker 임베딩 캐싱 (단 1회)
zero_wav, zero_sr = torchaudio.load(os.path.join(AUDIO_PATH, zero_shot_ref))
zero_shot_speaker = model.make_speaker_embedding(zero_wav, zero_sr)

few_embeddings = []
for file in few_shot_refs:
    wav, sr = torchaudio.load(os.path.join(AUDIO_PATH, file))
    emb = model.make_speaker_embedding(wav, sr)
    few_embeddings.append(emb)
few_shot_speaker = torch.stack(few_embeddings).mean(dim=0)

# 🎙 Zero-Shot 합성
def zero_shot(text, name):
    torch.cuda.empty_cache()
    cond = make_cond_dict(text=text, speaker=zero_shot_speaker, language="ko")
    conditioning = model.prepare_conditioning(cond)
    codes = model.generate(conditioning, disable_torch_compile=True)
    wavs = model.autoencoder.decode(codes).cpu()
    torchaudio.save(os.path.join(AUDIO_PATH, f"{name}_zero_shot.wav"), wavs[0], model.autoencoder.sampling_rate)

# 🎙 Few-Shot 합성
def few_shot(text, name):
    torch.cuda.empty_cache()
    cond = make_cond_dict(text=text, speaker=few_shot_speaker, language="ko")
    conditioning = model.prepare_conditioning(cond)
    codes = model.generate(conditioning, disable_torch_compile=True)
    wavs = model.autoencoder.decode(codes).cpu()
    torchaudio.save(os.path.join(AUDIO_PATH, f"{name}_few_shot.wav"), wavs[0], model.autoencoder.sampling_rate)

# 🕒 합성 실행 및 시간 측정
results = []
for idx, text in enumerate(pages):
    page_name = f"page_{idx}"
    print(f"\n🎤 처리 중: {page_name}")

    # Zero-Shot
    start = time.time()
    try:
        zero_shot(text, page_name)
        zero_time = round(time.time() - start, 2)
        print(f"✅ Zero-Shot 완료: {zero_time}s")
    except Exception as e:
        zero_time = f"Error: {str(e)}"

    # Few-Shot
    start = time.time()
    try:
        few_shot(text, page_name)
        few_time = round(time.time() - start, 2)
        print(f"✅ Few-Shot 완료: {few_time}s")
    except Exception as e:
        few_time = f"Error: {str(e)}"

    results.append({
        "page": page_name,
        "zero_shot": zero_time,
        "few_shot": few_time
    })

# 📊 결과 출력
df = pd.DataFrame(results)
print("\n⏱️ 페이지별 생성 시간:\n")
print(tabulate(df, headers='keys', tablefmt='github', showindex=False))

# ZONOS 웹에서 gradio로 부르기

In [ ]:
from gradio_client import Client
from scipy.io.wavfile import write as write_wav
import numpy as np
import soundfile as sf  # for saving the audio
import numpy as np
# 1. Gradio 앱 URL (예: 로컬 또는 share URL)
client = Client("http://localhost:7860")  # 또는 "https://xxxx.gradio.live"

# 2. generate_audio에 맞게 입력값 설정
result = client.predict(
    "Zyphra/Zonos-v0.1-transformer",  # model_choice
    "안녕하세요. 반갑습니다. 오늘 수업할 내용을 말씀드리겠습니다.",            # text
    "ko",                          # language
    None,                             # speaker_audio (None = 사용 안 함)
    None,                             # prefix_audio (None = 사용 안 함)
    1.0, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.2,  # emotion sliders
    0.78,                             # vq_single
    24000,                            # fmax
    45.0,                             # pitch_std
    15.0,                             # speaking_rate
    4.0,                              # dnsmos
    False,                            # speaker_noised
    2.0,                              # cfg_scale
    0,                              # top_p
    0,                                # top_k (min_k로 전달됨)
    0,                              # min_p
    0.5, 0.4, 0.0,                    # linear, confidence, quadratic
    42,                               # seed
    False,                            # randomize_seed
    ["emotion"],                     # unconditional_keys
    api_name="/generate_audio"              # 예: 함수가 자동 등록된 이름
)


import shutil

src = result[0]
dst = "test.wav"

shutil.copyfile(src, dst)

# FASTAPI & STREAMLIT

In [ ]:
# utils.py
import re
from typing import List
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from langchain_openai import OpenAIEmbeddings

def extract_top_similar_words(text: str, keywords: List[str], top_k: int = 10) -> List[str]:
    # 텍스트에서 단어 토큰 추출
    words = re.findall(r'\w+', text)
    word_counts = Counter(words)
    unique_words = list(word_counts.keys())

    # 임베딩
    embedder = OpenAIEmbeddings()
    word_embeddings = embedder.embed_documents(unique_words)
    keyword_embeddings = [embedder.embed_query(k) for k in keywords]

    # 코사인 유사도 계산
    similarities = {}
    for word, emb in zip(unique_words, word_embeddings):
        sim_scores = [cosine_similarity([emb], [k_emb])[0][0] for k_emb in keyword_embeddings]
        similarities[word] = max(sim_scores)

    # 상위 N개 단어 반환
    top_similar = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:top_k]
    return [{"word": word, "similarity": round(score, 4)} for word, score in top_similar]


In [ ]:
# script_generate.py
from utils import extract_top_similar_words

def combine_scripts_and_extract_keywords(scripts: List[str], keywords: List[str]):
    """전체 대본 통합 및 유사 단어 추출"""
    full_script = "\n\n".join(scripts)
    similar_words = extract_top_similar_words(full_script, keywords, top_k=10)
    return {
        "full_script": full_script,
        "top_similar_words": similar_words
    }


In [ ]:
# routes.py
from fastapi import APIRouter, Body
from typing import List
from core.script_generate import combine_scripts_and_extract_keywords

@router.post("/script/combine-and-analyze")
async def combine_and_analyze(
    scripts: List[str] = Body(...),
    keywords: List[str] = Body(...)
):  
    try:
        result = combine_scripts_and_extract_keywords(scripts, keywords)
        return result
    except Exception as e:
        return {"error": str(e)}
